In [1]:
import os
import sys
sys.path.append('/root/vr-logger')
import numpy as np

In [2]:



def set_cyclonedds_config():
    # Generate the XML configuration
    xml_content = f"""
      <CycloneDDS>
      <Domain id="any">
              <General>
                  <NetworkInterfaceAddress>192.168.2.123</NetworkInterfaceAddress>
              </General>
          <Discovery>
              <Peers>
                  <Peer address="192.168.2.123"/>
              </Peers>
              <ParticipantIndex>auto</ParticipantIndex>
          </Discovery>
          </Domain>
      </CycloneDDS>
    """

    # Create a temporary XML file
    cfg_file_path = os.path.join('cyclonedds.xml')
    with open(cfg_file_path, 'w') as f:
        f.write(xml_content)
    
    # Set the environment variable
    os.environ['CYCLONEDDS_URI'] = f'file://{cfg_file_path}'
# set_cyclonedds_config()

In [3]:
import os
def set_cyclonedds_config(interface_name):
    # Generate the XML configuration
    xml_content = f"""<CycloneDDS>
    <Domain>
        <General>
        <Interfaces>
            <NetworkInterface name="{interface_name}" priority="default" multicast="default" />
        </Interfaces>
        </General>
    </Domain>
    </CycloneDDS>"""

    # Create a temporary XML file
    cfg_file_path = os.path.join('/home/cyclonedds.xml')
    with open(cfg_file_path, 'w') as f:
        f.write(xml_content)
    
    # Set the environment variable
    os.environ['CYCLONEDDS_URI'] = f'file://{cfg_file_path}'

In [4]:
set_cyclonedds_config('enp4s0')

In [5]:

from threading import Thread
from VR_DDS_Logger_python.dds.PoseMsg import VRPose, Pose6D
from VR_DDS_Logger_python.dds.telemetry import Subscriber
from cyclonedds.domain import DomainParticipant
from cyclonedds.topic import Topic
from cyclonedds.sub import DataReader
from cyclonedds.pub import DataWriter
from cyclonedds.util import duration
from threading import Thread


from VR_DDS_Logger_python.dds.FlexivState import FlexivStateMsg
from VR_DDS_Logger_python.dds.telemetry import Publisher

class VRBridge:
    def __init__(self):
        self.participant = DomainParticipant()
        self.vr_pose_topic = Topic(self.participant, 'vr_poses_msg', VRPose)
        self.vr_pose_reader = DataReader(self.participant, self.vr_pose_topic)
        self.tool_pose_topic = Topic(self.participant, 't_tool_pose', Pose6D)
        self.tool_pose_reader = DataReader(self.participant, self.tool_pose_topic)
        self.running = True
        self.vr_pose = None
        self.tool_pose = None
        self.vr_thread = Thread(target=self.update_vr_pose)
        self.tool_thread = Thread(target = self.update_tool_pose)
        self.vr_thread.start()
        self.tool_thread.start()
        self.vr_joint_publisher = Publisher(FlexivStateMsg, 'isaac_flexiv_joint_cmds')
        
    def update_vr_pose(self):
        while self.running:
            for msg in self.vr_pose_reader.take_iter(timeout=duration(milliseconds=5.)):
                self.vr_pose = msg

    def update_tool_pose(self):
        while self.running:
            for msg in self.tool_pose_reader.take_iter(timeout=duration(milliseconds=5.)):
                self.tool_pose = msg

    def send_robot_state(self, q):
      if isinstance(q, np.ndarray):
         q = q.tolist()
      vr_joint_state = FlexivStateMsg(q= q, dq = 7*[0], tau= 7*[0])
      self.vr_joint_publisher.send(vr_joint_state)

    def close(self):
        self.running = False

In [6]:
vr_listener = VRBridge()

In [8]:
vr_listener.tool_pose

Pose6D(orientation=[0.0, 0.0, 0.0, 1.0], translation=[5.9604644775390625e-06, -7.897615432739258e-06, 3.5762786865234375e-05])

In [9]:
from FlexivPy.robot.model.pinocchio import FlexivModel
from FlexivPy.sim.MuJoCo import FlexivSimMujoco
from FlexivPy.controllers.taskspace import DiffIKController
model = FlexivModel()
robot = FlexivSimMujoco(mode='velocity', render = True, dt = 0.01)

libGL error: glx: failed to create dri3 screen
libGL error: failed to load driver: nouveau


robot sim is ready!


In [10]:
push_t_q0 = np.array(
            [
            0.001961823320016265,
            -0.20705948770046234,
            0.21981844305992126,
            2.069606304168701,
            -0.0594966784119606,
            0.6976734399795532,
            0.07441084086894989,
        ]
)

In [11]:
diffik_controller = DiffIKController(model, dt=0.01)

In [12]:
def mapControl(cmd, hand_center, robot_center, hand_range, robot_range):
    x_max = hand_center[0] + hand_range[0]
    x_min = hand_center[0] - hand_range[0]
    y_max = hand_center[1] + hand_range[1]
    y_min = hand_center[1] - hand_range[1]

    if cmd[0] > x_max or cmd[0] < x_min or cmd[1] > y_max or cmd[1] < y_min:
        return None
    else:
        mx = robot_range[0]/hand_range[0]
        my = robot_range[1]/hand_range[1]
        cx_robot = robot_center[0]
        cy_robot = robot_center[1]
        cx_hand = hand_center[0]
        cy_hand = hand_center[1]
        bx = cx_robot - mx*cx_hand
        by = cy_robot - my*cy_hand
        cmd_x = mx*cmd[0] + bx
        cmd_y = my*cmd[1] + by
        return [cmd_x, cmd_y]


In [13]:
robot_center = np.array([0.55, 0.0])
hand_center = np.array([0.73, 0.0])
robot_range = np.array([1.0, 1.0])
hand_range = np.array([0.5, 0.5])
mapControl([0.73, 0.0], hand_center, robot_center, hand_range, robot_range)

[0.55, 0.0]

In [25]:
import time
import numpy as np
import datetime
from copy import deepcopy

# Reset the robot and the controller
robot.reset_state_robot(push_t_q0, np.zeros_like(model.q0))
state = robot.get_robot_state()
q = np.array(state.q)
dq = np.array(state.dq)
info = model.getInfo(q, dq)
T0_cmd = info['poses']['link7'].copy()
diffik_controller.set_target(T0_cmd)
# Initialize the interaction data
dataset = {}
t_tool_qs = []
t_tool_ts = []
robot_qs = []
robot_dqs = []
vr_cmds = []
stamps = []

R0 = np.diag([1, -1, -1])
cmd_min = [0.35, -0.4, 0.07]
cmd_max = [0.75, 0.4, 0.8]

start_time = time.time()
while time.time()-start_time < 35*60:
    time.sleep(0.01)
    vr_cmd = vr_listener.vr_pose
    t_tool_pose = vr_listener.tool_pose
    robot_state = robot.get_robot_state()
    if robot_state is not None:
        vr_listener.send_robot_state(robot_state.q)
    if vr_cmd is not None and t_tool_pose is not None:
        T_cmd = T0_cmd.copy()   

        clipped_cmd = mapControl(vr_cmd.right_t, hand_center, robot_center, hand_range, robot_range) 
        if clipped_cmd is not None:
            vr_command = clipped_cmd + [0.61]
            T_cmd[:3, -1] = np.clip(np.array(vr_command), cmd_min, cmd_max)
            # T_cmd[2, -1] = 0.61
            # T_cmd[2, -1] +=0.55
        # Apply the control
        diffik_controller.set_target(T_cmd)
        s = robot.get_robot_state()
        cmd = diffik_controller.get_control(s, 0.0)
        robot.set_cmd(cmd)
        robot.step()
        robot.viewer.sync()
        # Record the interaction data
        vr_cmds.append(T_cmd[:3, -1].copy().tolist())
        t_tool_ts.append(deepcopy(t_tool_pose.translation))
        t_tool_qs.append(deepcopy(t_tool_pose.orientation))
        robot_qs.append(robot_state.q)
        robot_dqs.append(robot_state.dq)
        stamps.append(time.time()-start_time)

In [26]:
len(t_tool_ts)

167294

In [27]:
t_tool_ts = np.array(t_tool_ts)
t_tool_qs = np.array(t_tool_qs)
robot_qs = np.array(robot_qs)
robot_dqs = np.array(robot_dqs)
vr_cmds = np.array(vr_cmds)
stamps = np.array(stamps)

In [28]:
import pickle
import datetime

exp_name = 't-tool-push-2d'
exp_date = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
exp_dir = os.path.join('data', exp_name, exp_date)
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

with open(os.path.join(exp_dir, 'data.pkl'), 'wb') as f:
    pickle.dump(
        dict(
            t_tool_ts = t_tool_ts, 
            t_tool_qs = t_tool_qs, 
            robot_qs = robot_qs,
            robot_dqs = robot_dqs, 
            vr_cmds = vr_cmds, 
            stamps = stamps,
        ), f
    )

In [ ]:
vr_listener.close()